In [23]:
import tensorflow as tf

In [24]:
from tensorflow.examples.tutorials.mnist import input_data

In [25]:
mnist = input_data.read_data_sets('./data/MNIST_data/', one_hot=True)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

In [27]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [28]:
def conv2d(x,W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding='SAME')

In [29]:
def max_pool_2by2(x):
    #x --> [batch, h, w, c]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [30]:
#conv layer
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W)+b)

In [31]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b    

In [32]:
#placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [33]:
#layers
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [34]:
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [35]:
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [36]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])

In [37]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [38]:
#dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [39]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [40]:
#LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [41]:
#OPTMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [42]:
init = tf.global_variables_initializer()

In [43]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict = {x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print('ON STEP: {}'. format(i))
            print('ACCURACY: ')
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))

ON STEP: 0
ACCURACY: 
0.1199
ON STEP: 100
ACCURACY: 
0.9427
ON STEP: 200
ACCURACY: 
0.959
ON STEP: 300
ACCURACY: 
0.9719
ON STEP: 400
ACCURACY: 
0.9683
ON STEP: 500
ACCURACY: 
0.9768
ON STEP: 600
ACCURACY: 
0.9787
ON STEP: 700
ACCURACY: 
0.9756
ON STEP: 800
ACCURACY: 
0.9837
ON STEP: 900
ACCURACY: 
0.9809
ON STEP: 1000
ACCURACY: 
0.9789
ON STEP: 1100
ACCURACY: 
0.9847
ON STEP: 1200
ACCURACY: 
0.9842
ON STEP: 1300
ACCURACY: 
0.9821
ON STEP: 1400
ACCURACY: 
0.9857
ON STEP: 1500
ACCURACY: 
0.988
ON STEP: 1600
ACCURACY: 
0.986
ON STEP: 1700
ACCURACY: 
0.9884
ON STEP: 1800
ACCURACY: 
0.9885
ON STEP: 1900
ACCURACY: 
0.9862
ON STEP: 2000
ACCURACY: 
0.9873
ON STEP: 2100
ACCURACY: 
0.9881
ON STEP: 2200
ACCURACY: 
0.9885
ON STEP: 2300
ACCURACY: 
0.9877
ON STEP: 2400
ACCURACY: 
0.9899
ON STEP: 2500
ACCURACY: 
0.9876
ON STEP: 2600
ACCURACY: 
0.9898
ON STEP: 2700
ACCURACY: 
0.9867
ON STEP: 2800
ACCURACY: 
0.9891
ON STEP: 2900
ACCURACY: 
0.9887
ON STEP: 3000
ACCURACY: 
0.9877
ON STEP: 3100
ACCURACY:

In [44]:
print(batch_x)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [47]:
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
batch_x.shape

(50, 784)